# 6.838, Shape Analysis:  Homework 2

All homeworks will be graded out of 100 points.
<hr/>
For this homework, we will need a few convenience methods for displaying meshes textured with one value per vertex.  For convenience, those methods are displayed in the next block.  Please run it before starting your homework, so that you can use the methods in the problems.

In [17]:
import numpy as np
import trimesh
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import matplotlib.cm as cm

def map_z2color(zval, colormap, vmin, vmax):
    t=(zval-vmin)/float((vmax-vmin)); R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+','+'{:d}'.format(int(B*255+0.5))+')'   

def tri_indices(simplices):
    return ([triplet[c] for triplet in simplices] for c in range(3))

def plotly_trisurf(x, y, z, colors, simplices, colormap=cm.RdBu, plot_edges=False):
    
    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)
    
    ncolors = (colors-np.min(colors))/(np.max(colors)-np.min(colors))
    
    I,J,K=tri_indices(simplices)
    triangles=Mesh3d(x=x,y=y,z=z,
                     intensity=ncolors,
                     colorscale='Viridis',
                     i=I,j=J,k=K,name='',
                     showscale=True,
                     colorbar=ColorBar(tickmode='array', 
                                       tickvals=[np.min(z), np.max(z)], 
                                       ticktext=['{:.3f}'.format(np.min(colors)), 
                                                 '{:.3f}'.format(np.max(colors))]))
    
    if plot_edges is False: # the triangle sides are not plotted
        return Data([triangles])
    else:
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None] for T in tri_vertices] for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]
        lines=Scatter3d(x=Xe,y=Ye,z=Ze,mode='lines',line=Line(color='rgb(50,50,50)', width=1.5))
        return Data([triangles, lines])

def textured_mesh(mesh, per_vertex_signal, filename):
    x = mesh.vertices.transpose()[0]; y = mesh.vertices.transpose()[1]; z = mesh.vertices.transpose()[2];
    data1 = plotly_trisurf(x, y, z, per_vertex_signal, mesh.faces, colormap=cm.RdBu,plot_edges=True)
    axis = dict(showbackground=True,backgroundcolor="rgb(230, 230,230)",gridcolor="rgb(255, 255, 255)",zerolinecolor="rgb(255, 255, 255)")
    layout = Layout(width=800, height=800,scene=Scene(xaxis=XAxis(axis),yaxis=YAxis(axis),zaxis=ZAxis(axis),aspectratio=dict(x=1,y=1,z=1)))
    fig1 = Figure(data=data1, layout=layout)
    iplot(fig1, filename=filename)

<b>Problem 1:  Gaussian curvature on meshes (30 points)</b><br/>
We provide code below for loading a triangle mesh and displaying one value per vertex.  Please implement an approximation of vertex-based Gaussian curvature on the mesh and add a few sentences analyzing the results.  You can load different meshes or edit the vertex positions to better illustrate when your approximation is successful.  You can implement any approximation of your choosing.

In [18]:
mesh = trimesh.load_mesh('moomoo.off')

# For convenience, collect matrices of vertices (nv x 3) and triangles (nt x 3)
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices
nv = X.shape[0] # number of vertices
nt = T.shape[0] #number of triangles

# WRITE CODE TO COMPUTE CURVATURE HERE ############################################
gaussian_curvature = np.zeros(nv)
###################################################################################

textured_mesh(mesh,gaussian_curvature,'gauss')

<hr/>
<b>Problem 2:  Boundary and co-boundary operators (30 points)</b><br/>
<p>In class, we discussed "boundary operators" $\partial$ for an oriented triangle mesh with vertices $V$, edges $E$, and triangles $T$.  While finding the boundary of a simplex takes you down one dimension (from triangles to edges to vertices), we can define an operator $d$ that does the opposite.  In particular, $d$ will take one value per $k$-simplex and return one value per $(k+1)$-simplex (a triangle is a $2$-simplex, an edge is a $1$-simplex, and a vertex is a $0$-simplex).  Think of edges as <i>ordered</i> tuples $(v_1,v_2)$ and triangles as <i>ordered</i> triples $(v_1,v_2,v_3)$, where $v_i\in V$.</p>
<p><b>a (15 points).  Define $d_{0\rightarrow1}\in\{-1,0,1\}^{|E|\times|V|}$ via</b>
$$
(d_{0\rightarrow1})_{ev}=\left\{
\begin{array}{ll}
-1 & \textrm{ if }e=(v,w)\textrm{ for some }w\in V\\
1 & \textrm{ if }e=(w,v)\textrm{ for some }w\in V\\
0 &\textrm { otherwise.}
\end{array}
\right.
$$
<b>Similarly, define $d_{1\rightarrow 2}\in\{-1,0,1\}^{|T|\times |E|}$ via</b>
$$(d_{1\rightarrow2})_{te}=\left\{
\begin{array}{ll}
-1 & \textrm{ if triangle $t$ and edge $e$ have opposite orientation}\\
1 & \textrm{ if triangle $t$ and edge $e$ are consistently oriented}\\
0 &\textrm { if edge $e$ is not in triangle $t$.}
\end{array}
\right.
$$
<b>Show that $d_{1\rightarrow2}d_{0\rightarrow1}=0$.</b>
</p>

<b>Answer:</b> 

<b>b (15 points).  Complete the code below to compute matrices <code>d01</code> and <code>d12</code>.</b>  <br/><i>Note:</i> To receive credit, these matrices must be sparse.  The given list of triangles is consistently oriented; you may orient edges any way you want.  In the mesh <code>moomoo.off</code>, there are 3129 edges (this should be a dimension in both your matrices, <i>not</i> two times this number!).
<p><i>Hint:</i>  For <code>moomoo.off</code>, $d_{0\rightarrow1}\in\mathbb{R}^{3129\times1045}$ and $d_{1\rightarrow2}\in\mathbb{R}^{2086\times3129}$.  Check out <code>csr_matrix</code> in Python for constructing sparse matrices.</p>

In [19]:
from scipy.sparse import csr_matrix
from scipy.sparse import linalg as LA

mesh = trimesh.load_mesh('moomoo.off')

# For convenience, collect matrices of vertices (nv x 3) and triangles (nt x 3)
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices
nv = X.shape[0] # number of vertices
nt = T.shape[0] #number of triangles

# ADD CODE FOR COMPUTING d01 AND d12 HERE ##################

# Suggestion:  Similar to X and T above, make an ne x 2 matrix E of (unique) edges

# SOME SANITY CHECKS #######################################

#Commented out because you need to add code for computing the d's first
#print('This should equal zero:  ' + str(LA.norm(d12*d01)))
#print('d01 number of nonzeros = ' + str(d01.nnz) + ' (should equal ' + str(3*nt) + ')')
#print('d12 number of nonzeros = ' + str(d12.nnz) + ' (should equal ' + str(3*nt) + ')')

<hr/>
<b>Problem 3:  Mean curvature (40 points)</b>

<b>a (5 points).  Complete the function <code>surface_area</code>, which given the vertices and triangles of a triangle mesh returns its surface area.</b>

In [20]:
def surface_area(X,T): # takes matrix of vertices and matrix of triangles
    # ADD CODE FOR COMPUTING SURFACE AREA HERE #################
    return 0

# Testing code below
filename = 'moomoo.off'
mesh = trimesh.load_mesh(filename)

# For convenience, collect matrices of vertices (nv x 3) and triangles (nt x 3)
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices

print('The surface area of ' + filename + ' is ' + str(surface_area(X,T)) + '.')

The surface area of moomoo.off is 0.


<b>b (10 points).  Complete the function <code>cot_laplacian</code>, which returns a <i><u>sparse</u></i> matrix <code>L</code> such that $\nabla_p A=\frac{1}{2}L\cdot p$, where $A$ is the surface area, $p\in\mathbb{R}^{|V|\times3}$ contains vertex positions, and $L\in\mathbb{R}^{|V|\times|V|}$ depends on $p$ and the topology of the mesh.</b>
<p><i>Hint:</i>   If $\theta$ is the angle between vectors $v$ and $w$, then $\cot\theta=\frac{v\cdot w}{\|v\times w\|}$ (make sure you understand why!).</p>

In [21]:
from scipy.sparse import spdiags
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs

def cot_laplacian(X,T):
    # ADD CODE FOR COMPUTING L HERE ############################
    return csr_matrix((X.shape[0],X.shape[0]))
        
# Testing code below
L = cot_laplacian(X,T)

# A few sanity checks for your convenience
eigenvalues,eigenvectors = eigs(L,k=10,which='SR')
print('L should be symmetric and semidefinite:')
print(eigenvalues)
print(LA.norm(L-L.transpose()))

L should be symmetric and semidefinite:
[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j]
0.0


<b>c (5 points).  Write code to verify $\nabla_p A=\frac{1}{2}L\cdot p$ for your choice of $L$.  In particular, approximate $\nabla_p A$ using divided differences and show that the error between this approximation and the true gradient you computed in part (b) is small.</b>

<p><i>Hint:</i> Use the approximation $f'(x)\approx \frac{1}{2h}(f(x+h)-f(x-h))$.</p>

In [22]:
h = 1e-6

gradApprox = np.zeros((nv,3))

for i in range(0,nv):
    for j in range(0,3):
        # ADD CODE FOR COMPUTING gradApprox HERE ###################
        gradApprox[i,j] = 0
        
L = cot_laplacian(X,T)

print('These values should be all close to 1:')
print(gradApprox/(.5*L*X))

These values should be all close to 1:
[[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]
 ..., 
 [ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]


<b>d (10 points).  The <i>barycentric area</i> associated to a vertex is $\frac{1}{3}$ times the sum of triangle areas adjacent to that vertex.  Complete the function <code>barycentricArea</code> below, argue that the sum of barycentric areas is the surface area, and check that this is the case for your implementation.</b>

In [23]:
def barycentric_areas(X,T):
    # ADD CODE FOR COMPUTING BARYCENTRIC AREA VECTOR HERE ######
    return np.zeros(X.shape[0])

<b>Mathematical idea:</b> <i>Write me!</i>

<b>e (10 points).  Combine the methods above to approximate a per-vertex pointwise (non-integrated!) mean curvature on the mesh.</b>

In [24]:
unitNormals = mesh.vertex_normals; # may be useful for recovering sign of H

# ADD CODE FOR COMPUTING MEAN CURVATURE HERE ###############
H = np.zeros(X.shape[0])

textured_mesh(mesh,H,'mean')

<hr/>
<b>Extra credit (5 points)</b>

<p>The slides on discrete Gaussian curvature leave a small lemma to "homework," that $\sum_j\theta_j=\sum_j\varepsilon_j$.  Prove this lemma below.</p>

<b>Answer:</b>

<hr/>
<b>Extra credit (10 points; 20 for truly exceptional work)</b>

<p>Every 6.838 assignment can be accompanied with <i>open-ended</i> extra credit.  Excited about an algorithm we covered in class?  Implement it in here!  Curious about whether ideas from shape analysis can be applied to a particular field?  Give it a try.  Write it up below, or share a second document and tell us where to find it.</p>